In [2]:
import os
import numpy as np
import pandas as pd

from src import RESULT_DIR
from src.utils import MCC, DIS

In [3]:
classifier = 'RandomForest' # RandomForest, LightGBM
alignment = 'temporal' # geometric, temporal, warp

In [7]:
if classifier == 'LightGBM': 
        cols = ['fold_num', 'trial', 'split_num', 'num_leaves', 
                'min_child_samples', 'threshold', 'num_trees', 
                'tn', 'fp', 'fn', 'tp','time']
        cc = ['fold_num', 'trial', 'num_leaves', 'min_child_samples', 
              'threshold']
        
elif classifier == 'RandomForest': 
        cols = ['fold', 'step', 'threshold', 'voting_depth', 
                'voting_width', 'opening', 'closing', 'count',
                'tn', 'fp', 'fn', 'tp','time']
        cc = ['fold', 'step', 'threshold', 'voting_depth', 
                'voting_width', 'opening', 'closing', 'count']

In [16]:
res_dir = os.path.join(RESULT_DIR, classifier, alignment, 'postprocessing')
res_df = {}
res_mcc = []
for fold in range(1, 10):
    csv_dir = os.path.join(res_dir, 'results_post_fold{0:02d}.csv'.format(fold))
    dd = pd.read_csv(csv_dir, index_col=0)
    dd = dd[cols]
    dd.reset_index(drop=True, inplace=True)
    dd = dd.groupby(cc).aggregate({
                    'tn':np.sum,'fp':np.sum,'fn':np.sum,
                    'tp':np.sum,'time':np.average}).reset_index()
    dd['MCC'] = dd.apply(lambda x: MCC(x['tn'],x['fp'],x['fn'],x['tp']), axis=1)
    dd_max = dd.loc[dd['MCC'] == dd['MCC'].max(), :]
    print('Fold {0}:  {1:>3} iterations - max it. = {2:>3}, max.MCC = {3:.4f}  [{4:0.2f}  {5}  {6}  {7}  {8}  {9:0.2f}]'.format(fold,
                                                                                                                            dd[cc[1]].max()+1,
                                                                                                                            dd_max[cc[1]].values[0],
                                                                                                                            dd['MCC'].max(),
                                                                                                                            dd_max['threshold'].values[0]/100,
                                                                                                                            dd_max['voting_depth'].values[0],
                                                                                                                            dd_max['voting_width'].values[0],
                                                                                                                            dd_max['opening'].values[0],
                                                                                                                            dd_max['closing'].values[0],
                                                                                                                            dd_max['count'].values[0]/100,))
dd_max

Fold 1:  175 iterations - max it. = 163, max.MCC = 0.7598  [0.34  58  12  12  31  0.36]
Fold 2:   35 iterations - max it. =  19, max.MCC = 0.6227  [0.34  46  59  15  57  0.27]
Fold 3:   84 iterations - max it. =  81, max.MCC = 0.6519  [0.29  65  17  16  13  0.21]
Fold 4:  158 iterations - max it. = 120, max.MCC = 0.7803  [0.44  69  13  8  36  0.34]
Fold 5:  202 iterations - max it. = 192, max.MCC = 0.7264  [0.34  21  27  12  7  0.31]
Fold 6:   35 iterations - max it. =  19, max.MCC = 0.6634  [0.34  46  59  15  57  0.27]
Fold 7:  125 iterations - max it. = 110, max.MCC = 0.7664  [0.34  40  14  11  1  0.39]
Fold 8:  160 iterations - max it. = 155, max.MCC = 0.8127  [0.27  49  12  14  11  0.34]
Fold 9:   37 iterations - max it. =  19, max.MCC = 0.6430  [0.34  46  59  15  57  0.27]


,fold,step,threshold,voting_depth,voting_width,opening,closing,count,tn,fp,fn,tp,time,MCC
19,9,19,34,46,59,15,57,27,401492767.0,1290440.0,6437206.0,6616034.0,36.746025,0.643037


In [31]:
res_dir = os.path.join(RESULT_DIR, classifier, alignment, 'postprocessing')
res_df = {}
res_mcc = []
for fold in range(1,10):
    csv_dir = os.path.join(res_dir, 'results_fold{0:02d}.csv'.format(fold))
    dd = pd.read_csv(csv_dir, index_col=0)
    dd = dd[cols]
    dd.reset_index(drop=True, inplace=True)
    dd = dd[dd[cc[2]] <=200]
    #dd['trial'] = np.repeat(list(range(int(dd.shape[0] / 8))),8).astype(int)
    dd = dd.groupby(cc).aggregate({
                    'tn':np.sum,'fp':np.sum,'fn':np.sum,
                    'tp':np.sum,'time':np.average}).reset_index()
    dd['MCC'] = dd.apply(lambda x: MCC(x['tn'],x['fp'],x['fn'],x['tp']), axis=1)
    dd_max = dd.loc[dd['MCC'] == dd['MCC'].max(), :]
    #res_df[fold] = {'num_leaves':dd_max['num_leaves'].values[0], 'min_child_samples':dd_max['min_child_samples'].values[0], 'threshold':dd_max['threshold'].values[0]}
    print('Fold {0}:  {1:>3} iterations - max it. = {6}, max.MCC = {2:.4f}  [{3}  {4}  {5:0.4f}]'.format(fold,dd[cc[1]].max()+1,dd['MCC'].max(),
                                                                                                         dd_max[cc[2]].values[0],
                                                                                                         dd_max[cc[3]].values[0],
                                                                                                         dd_max['threshold'].values[0],
                                                                                                         dd_max[cc[1]].values[0]))
    #res_mcc.append(dd['MCC'].max())


Fold 1:  298 iterations - max it. = 1, max.MCC = 0.8104  [116  175  18.0000]
Fold 2:  300 iterations - max it. = 1, max.MCC = 0.8173  [116  175  18.0000]
Fold 3:  300 iterations - max it. = 177, max.MCC = 0.8072  [140  200  16.0000]
Fold 4:  298 iterations - max it. = 221, max.MCC = 0.8301  [101  200  18.0000]
Fold 5:  300 iterations - max it. = 70, max.MCC = 0.8041  [116  200  22.0000]
Fold 6:  300 iterations - max it. = 232, max.MCC = 0.8264  [172  200  19.0000]
Fold 7:  300 iterations - max it. = 270, max.MCC = 0.8505  [159  179  18.0000]
Fold 8:  300 iterations - max it. = 294, max.MCC = 0.8133  [98  200  16.0000]
Fold 9:  300 iterations - max it. = 98, max.MCC = 0.8232  [68  200  21.0000]


In [5]:
Fold 1:  300 iterations - max it. = 204, max.MCC = 0.8034  [100  200  16.0000]
Fold 2:  300 iterations - max it. = 285, max.MCC = 0.8123  [79  69  17.0000]
Fold 3:  300 iterations - max it. = 158, max.MCC = 0.7973  [110  192  13.0000]
Fold 4:  300 iterations - max it. = 122, max.MCC = 0.8219  [77  196  17.0000]
Fold 5:  300 iterations - max it. = 144, max.MCC = 0.7974  [88  200  17.0000]
Fold 6:  300 iterations - max it. = 170, max.MCC = 0.8215  [130  199  18.0000]
Fold 7:  300 iterations - max it. = 176, max.MCC = 0.8452  [217  196  15.0000]
Fold 8:  300 iterations - max it. = 189, max.MCC = 0.8159  [195  158  14.0000]
Fold 9:  300 iterations - max it. = 188, max.MCC = 0.8080  [45  200  19.0000]


Fold 1:  300 iterations - max it. = 120, max.MCC = 0.7272  [200  46  38.0000]
Fold 2:  300 iterations - max it. = 143, max.MCC = 0.7370  [200  44  38.0000]
Fold 3:  300 iterations - max it. = 228, max.MCC = 0.7264  [200  67  37.0000]
Fold 4:  300 iterations - max it. = 81, max.MCC = 0.7517  [180  51  40.0000]
Fold 5:  159 iterations - max it. = 20, max.MCC = 0.7196  [200  165  40.0000]
Fold 6:  300 iterations - max it. = 163, max.MCC = 0.7494  [200  67  39.0000]
Fold 7:  300 iterations - max it. = 262, max.MCC = 0.7773  [199  117  39.0000]
Fold 8:  153 iterations - max it. = 76, max.MCC = 0.7395  [200  69  38.0000]
Fold 9:  241 iterations - max it. = 117, max.MCC = 0.7316  [200  43  38.0000]


Fold 1:  300 iterations - max it. = 1, max.MCC = 0.8104  [116  175  18.0000]
Fold 2:  300 iterations - max it. = 1, max.MCC = 0.8173  [116  175  18.0000]
Fold 3:  300 iterations - max it. = 177, max.MCC = 0.8072  [140  200  16.0000]
Fold 4:  300 iterations - max it. = 221, max.MCC = 0.8301  [101  200  18.0000]
Fold 5:  300 iterations - max it. = 70, max.MCC = 0.8041  [116  200  22.0000]
Fold 6:  300 iterations - max it. = 33, max.MCC = 0.8272  [214  200  17.0000]
Fold 7:  300 iterations - max it. = 270, max.MCC = 0.8505  [159  179  18.0000]
Fold 8:  300 iterations - max it. = 294, max.MCC = 0.8133  [98  200  16.0000]
Fold 9:  300 iterations - max it. = 98, max.MCC = 0.8232  [68  200  21.0000]

SyntaxError: invalid syntax (3534071284.py, line 1)

In [11]:
res_dir = os.path.join(RESULT_DIR, classifier, alignment, 'classification')
res_df = {}
res_mcc = []
for fold in range(1,10):
    csv_dir = os.path.join(res_dir, 'results_fold{0:02d}.csv'.format(fold))
    dd = pd.read_csv(csv_dir, index_col=0)
    dd = dd[cols]
    dd.reset_index(drop=True, inplace=True)
    #dd['trial'] = np.repeat(list(range(int(dd.shape[0] / 8))),8).astype(int)
    dd = dd.groupby(cc).aggregate({
                    'tn':np.sum,'fp':np.sum,'fn':np.sum,
                    'tp':np.sum,'time':np.average}).reset_index()
    dd['MCC'] = dd.apply(lambda x: MCC(x['tn'],x['fp'],x['fn'],x['tp']), axis=1)
    dd['DIS'] = dd.apply(lambda x: DIS(x['tn'],x['fp'],x['fn'],x['tp']), axis=1)
    dd_max = dd.loc[dd['MCC'] == dd['MCC'].max(), :]
    #res_df[fold] = {'num_leaves':dd_max['num_leaves'].values[0], 'min_child_samples':dd_max['min_child_samples'].values[0], 'threshold':dd_max['threshold'].values[0]}
    print('\\textbf{{{0}}} & {1:>3} & {2:>3} & {3:>3} & {4:0.2f} & {5:0.4f} & {6:0.4f} \\\\'.format(fold,
                                                                                            dd[cc[1]].max()+1,
                                                                                            dd_max[cc[2]].values[0],
                                                                                            dd_max[cc[3]].values[0],
                                                                                            dd_max['threshold'].values[0]/100,
                                                                                            dd_max['MCC'].values[0],
                                                                                            dd_max['DIS'].values[0],
                                                                                        ))

\textbf{1} & 288 & 200 &  50 & 0.39 & 0.7263 & 0.2003 \\
\textbf{2} & 300 & 200 &  44 & 0.39 & 0.7331 & 0.1940 \\
\textbf{3} & 300 & 200 &  68 & 0.38 & 0.7244 & 0.2001 \\
\textbf{4} & 300 & 200 &  50 & 0.40 & 0.7527 & 0.1856 \\
\textbf{5} & 284 & 200 &  78 & 0.41 & 0.7178 & 0.2151 \\
\textbf{6} & 300 & 200 &  73 & 0.40 & 0.7459 & 0.1830 \\
\textbf{7} & 300 & 200 &  57 & 0.40 & 0.7739 & 0.1640 \\
\textbf{8} & 287 & 200 &  52 & 0.38 & 0.7380 & 0.1908 \\
\textbf{9} & 300 & 200 &  64 & 0.39 & 0.7330 & 0.1991 \\


In [26]:
# Extra
import os
from src import DATA_DIR
from src.config import fold_split
import pickle

from torch.utils.data import DataLoader
from src.dataset import VDAODataset

alignment = 'temporal'

ddir = os.path.join(DATA_DIR, 'post')

for fold in range(1, 10):
    for split in range(1, 9):
        dfile = os.path.join(ddir, 
                            'data_rf_{0}_fold{1:02d}_spl{2:02d}.pkl'.format(alignment,
                                                                            fold,
                                                                            split))
        dd = pickle.load(open(dfile, 'rb'))
        files = list(dd['silhouette'].keys())
        dataset = VDAODataset(fold=fold,
                            split=split,
                            type='validation',
                            alignment=alignment,
                            transform=True)
        loader = DataLoader(dataset, num_workers=8, batch_size=1, shuffle=False)

        # List of videos
        videos = dataset.align_df.test_file.unique()
        print(fold, split, files.sort() == videos.sort())


1 1 True
1 2 True
1 3 True
1 4 True
1 5 True
1 6 True
1 7 True
1 8 True
2 1 True
2 2 True
2 3 True
2 4 True
2 5 True
2 6 True
2 7 True
2 8 True
3 1 True
3 2 True
3 3 True
3 4 True
3 5 True
3 6 True
3 7 True
3 8 True
4 1 True
4 2 True
4 3 True
4 4 True
4 5 True
4 6 True
4 7 True
4 8 True
5 1 True
5 2 True
5 3 True
5 4 True
5 5 True
5 6 True
5 7 True
5 8 True
6 1 True
6 2 True
6 3 True
6 4 True
6 5 True
6 6 True
6 7 True
6 8 True
7 1 True
7 2 True
7 3 True
7 4 True
7 5 True
7 6 True
7 7 True
7 8 True
8 1 True
8 2 True
8 3 True
8 4 True
8 5 True
8 6 True
8 7 True
8 8 True
9 1 True
9 2 True
9 3 True
9 4 True
9 5 True
9 6 True
9 7 True
9 8 True


In [24]:
fold_split[3][4]

{'training': [1, 4, 6, 7, 9], 'validation': [2, 5, 8]}